In [1]:
""" A CNN Model is trained and saved as SavedModel. It is then converted to TFlite model with optimizations
and finally downloaded to local storage. """

# Imports
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pathlib

In [2]:
splits = tfds.Split.ALL.subsplit(weighted=(80,10,10))
(train_examples,validation_examples,test_examples), info = tfds.load("fashion_mnist",with_info=True,
                                                                    as_supervised=True, split=splits)
num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

class_names = ['T-shirt_top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

with open('labels.txt','w') as f:
    f.write('\n'.join(class_names))

IMG_SIZE = 28

In [9]:
def format_image(image,label):
    image = tf.cast(image,tf.float32)
    image = tf.image.resize(image, (IMG_SIZE,IMG_SIZE)) / 255.0
    return image, label

BATCH_SIZE = 32

train_batches = train_examples.shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE)
test_batches = test_examples.map(format_image).batch(1)

In [11]:
# Creating model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 3, activation='relu', input_shape=(IMG_SIZE,IMG_SIZE,1)),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam', metrics=['accuracy'])
model.fit(train_batches, validation_data=validation_batches, epochs=1)

1750/1750 [==============================] - 65s 37ms/step - loss: 0.4330 - accuracy: 0.8457 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00


In [12]:
export_dir = 'saved_model/2'
tf.saved_model.save(model,export_dir)

# Converting model to tfLite model
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
# Optimizations
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/2/assets


INFO:tensorflow:Assets written to: saved_model/2/assets


In [13]:
tflite_model_file = 'model.tflite'

with open(tflite_model_file,'wb') as f:
    f.write(tflite_model)

In [17]:
# Interpreting
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]['index']
output_details = interpreter.get_output_details()[0]['index']

predictions = []
test_labels = []
test_images = []

for img, label in test_batches.take(50):
    interpreter.set_tensor(input_details,img)
    interpreter.invoke()
    predictions.append(interpreter.get_tensor(output_details))
    test_labels.append(label)
    test_images.append(np.array(img))

In [ ]:
# Finally downloading our tfLite model
from google.colab import files
files.download(tflite_model_file)
files.download('labels.txt')